In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os

In [11]:
# Define paths and parameters
data_dir = "/Users/mohamednashath/Desktop/projectface/datadir"  # Folder with subfolders: naathi, nazeef, riznee, unmatched
img_size = (224, 224)
batch_size = 16
num_classes = 4  # naathi, nazeef, riznee, unmatched
epochs = 30


In [12]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.1875  # 15% validation (84 images), 80% training (448 images)
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
# Load datasets
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # 448 images
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # 84 images
)
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # For consistent evaluation
)

Found 456 images belonging to 4 classes.
Found 104 images belonging to 4 classes.
Found 560 images belonging to 4 classes.


In [14]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


/opt/anaconda3/envs/cnn_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])



In [16]:
# Print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,907,908 (91.20 MB)

 Trainable params: 23,907,908 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

/opt/anaconda3/envs/cnn_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - Precision: 0.3262 - Recall: 0.2106 - accuracy: 0.2907 - loss: 4.4831 - val_Precision: 0.8000 - val_Recall: 0.1154 - val_accuracy: 0.6346 - val_loss: 1.1206
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - Precision: 0.6836 - Recall: 0.3161 - accuracy: 0.5680 - loss: 1.0467 - val_Precision: 0.9138 - val_Recall: 0.5096 - val_accuracy: 0.7692 - val_loss: 0.7373
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - Precision: 0.7205 - Recall: 0.5105 - accuracy: 0.6283 - loss: 0.8800 - val_Precision: 0.8876 - val_Recall: 0.7596 - val_accuracy: 0.8269 - val_loss: 0.4689
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - Precision: 0.7918 - Recall: 0.6555 - accuracy: 0.7272 - loss: 0.7350 - val_Precision: 0.9146 - val_Recall: 0.7212 - val_accuracy: 0.8654 - val_loss: 0.5227
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 150ms/step - Precision: 0.8243 - Recall: 0.6860 - accuracy: 0.7545 - loss: 0.6313 - val_Precision: 0.8913 - val_Recall:

In [18]:
# Evaluate on test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - Precision: 0.9953 - Recall: 0.9779 - accuracy: 0.9936 - loss: 0.0439
Test Accuracy: 0.9929, Precision: 0.9946, Recall: 0.9857


In [19]:
# Save model
model.save("face_recognition_cnn.h5")


In [25]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/naathi2.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted: riznee with probability 0.9998


In [26]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/naathi3.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted: naathi with probability 0.9968


In [24]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/naathi.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted: nazeef with probability 0.7817


In [27]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/naathi4.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted: naathi with probability 0.9680


In [28]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/naathi5.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted: riznee with probability 0.9838


In [29]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/nazeef.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicted: nazeef with probability 1.0000


In [30]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/sulaim.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted: naathi with probability 0.9820


In [31]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/riznee.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Not matching any known person


In [32]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/riznee2.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted: riznee with probability 0.9978


In [33]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/riznee3.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted: riznee with probability 0.9999


In [35]:
# Load model
model = tf.keras.models.load_model("face_recognition_cnn.h5")

# Test with an untrained image
from tensorflow.keras.preprocessing import image
def predict_image(img_path, model, threshold=0.5):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0]
    max_prob = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())
    
    if max_prob > threshold and class_idx != class_names.index('unmatched'):
        print(f"Predicted: {class_names[class_idx]} with probability {max_prob:.4f}")
    else:
        print("Not matching any known person")

# Example usage
predict_image("/Users/mohamednashath/Desktop/projectface/testrandom/riznee3.jpeg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted: riznee with probability 0.9999
